In [1]:
!pip -q install pyomo
!apt-get -q install -y -qq coinor-cbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3

In [2]:
from pyomo.environ import *
import numpy as np

In [3]:
# N: number of variables
# M: number of constraints

N, M = 11, 11

# vector of cost cofficient
c = np.array([3, 2, 6, 5, 5, 4, 6, 8, 8, 5, 4])    # credits of each course

# matrix of matrix for coefficient of constraints and vector of resource constraints
A = np.array([
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],   # sum(x_i) <= 9
              [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],   # x_2 + x_7 <= 1
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],   # x_4 + x_9 <= 1
              [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],   # x_2 + x_8 <= 1
              [-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],  # x_10 - x_0 <= 0
              [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],   # x_0 + x_1 + x_2 <= 2
              [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],   # x_5 + x_6 <= 1
              [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],   # x_3 + x_6 + x_9 <= 2

              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],   # sum(x_i) >= 5
              [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],   # x_0 + x_1 + x_2 >= 1
              c,                                   # sum(c_i*x_i) >= 30
              ])

b = np.array([9, 1, 1, 1, 0, 2, 1, 2, 5, 1, 30])

In [4]:
# defining empty pyomo model as ILP_Model
MILP_Model = ConcreteModel()

# variable are defined as NonNegativeIntegers
MILP_Model.x = Var(np.arange(N), domain = Binary, bounds= (0,1))

In [5]:
MILP_Model.constraints = ConstraintList()

[MILP_Model.constraints.add(sum(A[i][j]*MILP_Model.x[j] for j in range(N)) <= b[i]) for i in range(8)]

[MILP_Model.constraints.add(sum(A[i][j]*MILP_Model.x[j] for j in range(N)) >= b[i]) for i in range(8, 11)]

#[MILP_Model.constraints.add(sum(A[-1][j]*MILP_Model.x[j]*c[j] for j in range(N)) >= b[-1])]

In [6]:
# objective for above formulation defined
MILP_Model.Objective = Objective(expr = sum(c[j]*MILP_Model.x[j] for j in range(N)), sense = maximize)

# printing your formulation
MILP_Model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None 

In [7]:
optCBC = SolverFactory('cbc')

resultCBC = optCBC.solve(MILP_Model)
print(resultCBC)

def OptResults(Model):
  # display solution
  print('\nObjective = ', Model.Objective())

  print('\nDecision Variables')
  for i in np.arange(N):
    print('x[',i,'] = ', Model.x[i].value)
  print('\nConstraints')
  Model.constraints.display()

MILP_ModelSolution = OptResults(MILP_Model)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.04825782775878906
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Objective =  41.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0

Constraints
constraints : Size=11
    Ke

In [8]:
# Model
Linear_Model = ConcreteModel()

# Variables
Linear_Model.x = Var(np.arange(N), domain = Reals, bounds= (0,1))

# Constraints
Linear_Model.constraints = ConstraintList()

[Linear_Model.constraints.add(sum(A[i][j]*Linear_Model.x[j] for j in range(N)) <= b[i]) for i in range(8)]

[Linear_Model.constraints.add(sum(A[i][j]*Linear_Model.x[j] for j in range(N)) >= b[i]) for i in range(8, 11)]

# Objective
Linear_Model.Objective = Objective(expr = sum(c[j]*Linear_Model.x[j] for j in range(N)), sense = maximize)

# Print the built Model
Linear_Model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 :  None 

In [9]:
# Solve
optCBC = SolverFactory('cbc')

# Results
resultCBC = optCBC.solve(Linear_Model)
print(resultCBC)

# Value obtained for objective and constraints
Linear_Model_Solution = OptResults(Linear_Model)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.01557016372680664
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Objective =  41.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0

Constraints
constraints : Size=11
    Key : Lower : Body : Upper
      1 :  None :  8.0 :   9.0
      

In [10]:
# N: number of variables
# M: number of constraints

N, M = 12, 12

# vector of cost cofficient
c_new = np.append(c, 9)    # credits of each course

# Add new coulm for C11 coefficients
C11_col = np.zeros(A.shape[1])
C11_col = np.expand_dims(C11_col, 1)
A_new = np.hstack((A, C11_col))

# Add credits of C11 for sum(c_i*x_i) >= 30 constraint
A_new[-1, -1] = 9

# Add coeeficients of the new constraint
A_new = np.vstack((A_new, [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1]))

# Update b
b_new = np.append(b, 1)

In [11]:
# Define model
MILP_Model = ConcreteModel()

# variables
MILP_Model.x = Var(np.arange(N), domain = Binary, bounds= (0,1))

# Constraints
MILP_Model.constraints = ConstraintList()

[MILP_Model.constraints.add(sum(A_new[i][j]*MILP_Model.x[j] for j in range(N)) <= b_new[i]) for i in range(8)]

[MILP_Model.constraints.add(sum(A_new[i][j]*MILP_Model.x[j] for j in range(N)) >= b_new[i]) for i in range(8, 11)]

# new constraint for C11
[MILP_Model.constraints.add(sum(A_new[-1][j]*MILP_Model.x[j] for j in range(N)) <= b_new[-1])]

# Objective
MILP_Model.Objective = Objective(expr = sum(c_new[j]*MILP_Model.x[j] for j in range(N)), sense = maximize)

# Print the built Model
MILP_Model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 

In [12]:
# Solve
optCBC = SolverFactory('cbc')

# Results
resultCBC = optCBC.solve(MILP_Model)
print(resultCBC)

# Value obtained for objective and constraints
MILP_Model_Solution = OptResults(MILP_Model)


Problem: 
- Name: unknown
  Lower bound: 38.0
  Upper bound: 38.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 12
  Number of binary variables: 12
  Number of integer variables: 12
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.026142597198486328
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Objective =  38.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  1.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0

Constraints
constraints :

In [13]:
# Define model
Linear_Model = ConcreteModel()

# variables
Linear_Model.x = Var(np.arange(N), domain = Reals, bounds= (0,1))

# Constraints
Linear_Model.constraints = ConstraintList()

[Linear_Model.constraints.add(sum(A_new[i][j]*Linear_Model.x[j] for j in range(N)) <= b_new[i]) for i in range(8)]

[Linear_Model.constraints.add(sum(A_new[i][j]*Linear_Model.x[j] for j in range(N)) >= b_new[i]) for i in range(8, 11)]

# new constraint for C11
[Linear_Model.constraints.add(sum(A_new[-1][j]*Linear_Model.x[j] for j in range(N)) <= b_new[-1])]

# Objective
Linear_Model.Objective = Objective(expr = sum(c_new[j]*Linear_Model.x[j] for j in range(N)), sense = maximize)

# Print the built Model
Linear_Model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 

In [14]:
# Solve
optCBC = SolverFactory('cbc')

# Results
resultCBC = optCBC.solve(Linear_Model)
print(resultCBC)

# Value obtained for objective and constraints
Linear_Model_Solution = OptResults(Linear_Model)


Problem: 
- Name: unknown
  Lower bound: 38.0
  Upper bound: 38.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 12
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 4
  Error rc: 0
  Time: 0.013585805892944336
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Objective =  38.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  1.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0

Constraints
constraints : Size=12
    Key : Lower : Body : Upper
      1 :  None :  6.

In [16]:
#2
import pandas as pd

In [17]:
!apt-get -q install -y -qq glpk-utils
optLP = SolverFactory('glpk', execute='/usr/bin/glpsol')

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 120920 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [18]:
!apt-get -q install -y -qq coinor-cbc
optCBC = SolverFactory('cbc')

In [19]:
# model

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([59, 73])

Model2 = ConcreteModel()  #defining empty model

Model2.x = Var(np.arange(M), domain = NonNegativeIntegers)  #defining variable type

Model2.constraints = ConstraintList()  #defining empty constraints list

[Model2.constraints.add(sum(A[i][j]*Model2.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

Model2.Objective = Objective(expr = sum(c[j]*Model2.x[j] for j in range(M)), sense = maximize)   #objective function

Model2.pprint()  #print the model

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers
          4 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 

In [20]:
#solving the problem
result_Model2 = optCBC.solve(Model2)
print(result_Model2)


Problem: 
- Name: unknown
  Lower bound: 114.0
  Upper bound: 114.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 5
  Number of binary variables: 0
  Number of integer variables: 5
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.0226898193359375
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [21]:
# snip to code for a function to print optimal solution as an output
def OptResults(Model):
  # display solution
  print('\nObjective = ', Model.Objective())

  print('\nDecision Variables')
  for i in np.arange(M):                      #this M is not a local variable in the function, but an already defined variable in the previous cell
    print('x[',i,'] = ', Model.x[i].value)
  print('\nConstraints')
  Model.constraints.display()

In [22]:
#printing the solution

OptResults(Model2)


Objective =  114.0

Decision Variables
x[ 0 ] =  6.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body  : Upper
      1 :  None :  58.5 :  59.0
      2 :  None : 35.25 :  73.0


In [ ]:
#Q2.3 Relaxing the variables from integers to real values

In [23]:
# new_model

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([59, 73])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeReals)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

new_Model.pprint()  #print the model

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 14*x[4]

1 Cons

In [24]:
#solving the problem
result_new_Model = optCBC.solve(new_Model)
print(result_new_Model)


Problem: 
- Name: unknown
  Lower bound: 118.0
  Upper bound: 118.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 5
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 3
  Error rc: 0
  Time: 0.017848491668701172
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [25]:
# snip to code for a function to print optimal solution as an output
def OptResults(Model):
  # display solution
  print('\nObjective = ', Model.Objective())

  print('\nDecision Variables')
  for i in np.arange(M):                      #this M is not a local variable in the function, but an already defined variable in the previous cell
    print('x[',i,'] = ', Model.x[i].value)
  print('\nConstraints')
  Model.constraints.display()

In [26]:
OptResults(new_Model)


Objective =  117.9999996

Decision Variables
x[ 0 ] =  9.8333333
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body         : Upper
      1 :  None :   58.9999998 :  59.0
      2 :  None : 41.791666525 :  73.0


In [ ]:
#Q2.5 Changing the RHS of the constraints in both MILP and LP

In [27]:
# for MILP

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([71, 87])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeIntegers)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

In [28]:
OptResults(new_Model)


Objective =  138.0

Decision Variables
x[ 0 ] =  8.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body  : Upper
      1 :  None :  70.5 :  71.0
      2 :  None : 43.75 :  87.0


In [29]:
# for LP

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([71, 87])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeReals)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

In [30]:
OptResults(new_Model)


Objective =  141.999996

Decision Variables
x[ 0 ] =  11.833333
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body        : Upper
      1 :  None :   70.999998 :  71.0
      2 :  None : 50.29166525 :  87.0


In [ ]:
#2.6 For different values of RHS

In [31]:
# for MILP with budget and parking area as 69,88

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([69, 88])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeIntegers)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

OptResults(new_Model)


Objective =  136.0

Decision Variables
x[ 0 ] =  9.0
x[ 1 ] =  2.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body  : Upper
      1 :  None :  69.0 :  69.0
      2 :  None : 44.75 :  88.0


In [32]:
# for LP with budget and parking area as 69,88

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([69, 88])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeReals)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

OptResults(new_Model)


Objective =  138.0

Decision Variables
x[ 0 ] =  11.5
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body   : Upper
      1 :  None :   69.0 :  69.0
      2 :  None : 48.875 :  88.0


In [33]:
# for MILP with budget and parking area as 67,89

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([67, 89])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeIntegers)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

OptResults(new_Model)


Objective =  132.0

Decision Variables
x[ 0 ] =  11.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body  : Upper
      1 :  None :  66.0 :  67.0
      2 :  None : 46.75 :  89.0


In [34]:
# for LP with budget and parking area as 67,89

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([67, 89])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeReals)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

OptResults(new_Model)


Objective =  134.000004

Decision Variables
x[ 0 ] =  11.166667
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body        : Upper
      1 :  None :   67.000002 :  67.0
      2 :  None : 47.45833475 :  89.0


In [35]:
# for MILP with budget and parking area as 65,90

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([65, 90])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeIntegers)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

OptResults(new_Model)


Objective =  126.0

Decision Variables
x[ 0 ] =  7.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body : Upper
      1 :  None : 64.5 :  65.0
      2 :  None : 39.5 :  90.0


In [36]:
# for LP with budget and parking area as 65,90

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([65, 90])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeReals)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

OptResults(new_Model)


Objective =  129.999996

Decision Variables
x[ 0 ] =  10.833333
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body        : Upper
      1 :  None :   64.999998 :  65.0
      2 :  None : 46.04166525 :  90.0


In [37]:
# for MILP with budget and parking area as 63,91

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([63, 91])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeIntegers)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

OptResults(new_Model)


Objective =  124.0

Decision Variables
x[ 0 ] =  8.0
x[ 1 ] =  2.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body : Upper
      1 :  None : 63.0 :  63.0
      2 :  None : 40.5 :  91.0


In [38]:
# for LP with budget and parking area as 63,91

#variables
M = 5 #number of variables for planes
N = 2 #number of constraints

c=np.array([12, 14, 10, 15, 14]) #objective function
A=np.array([[6, 7.5, 9.5, 10, 12.5],
            [4.25, 3.25, 2.5, 3.45, 2.25]])

b=np.array([63, 91])

new_Model = ConcreteModel()  #defining empty model

new_Model.x = Var(np.arange(M), domain = NonNegativeReals)  #defining variable type

new_Model.constraints = ConstraintList()  #defining empty constraints list

[new_Model.constraints.add(sum(A[i][j]*new_Model.x[j] for j in range(M)) <= b[i]) for i in range(N)]  #constraints

new_Model.Objective = Objective(expr = sum(c[j]*new_Model.x[j] for j in range(M)), sense = maximize)   #objective function

#solving the problem
result_new_Model = optCBC.solve(new_Model)

OptResults(new_Model)


Objective =  126.0

Decision Variables
x[ 0 ] =  10.5
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0

Constraints
constraints : Size=2
    Key : Lower : Body   : Upper
      1 :  None :   63.0 :  63.0
      2 :  None : 44.625 :  91.0


In [39]:
#3.
# number of variable representing warehouse (M=8) and markets (N=8)
M, N = 8,8

# since variable are two dimensional ie., x_{ij}, therefore both row and column indicies are defined below
row_indicies = np.arange(M)
col_indicies = np.arange(N)

# matrix of cost cofficient
cost = np.array([[627, 617, 1270, 2072, 1415,2943, 1064, 4265], [159, 1432, 351, 3372, 237, 3373, 1456, 5059], [609, 1943, 77, 3365, 298, 3021, 1484, 4803], [2143, 671, 3876, 537, 1902, 1941, 1682, 3348],
                 [2742, 2089, 1931, 2199, 1226, 739, 691, 1128],[2053, 860, 2995, 68, 2069, 2065, 993, 3342],[656, 817, 1245, 2141, 1534, 3351, 1137, 3887],[195, 1862, 1331, 2921, 402, 2825, 563, 4441]])

warehouse_cap = np.array([185,350,405,280,105,75,425,270])
market_demand = np.array([175, 400, 550, 70, 650, 100, 65, 85])
# defining an empty model using pyomo
TransModel = ConcreteModel()

# defining varibles used in model x_{ij} (in total 8*8 = 64 variables)
TransModel.x = Var(row_indicies, col_indicies, domain= NonNegativeReals)

# setting up the objective function
TransModel.Objective = Objective(expr = sum(cost[i][j]*TransModel.x[i,j] for i in row_indicies for j in col_indicies), sense = minimize)

In [40]:
# below we define constraints based on available demand at hub and capacity at plant
# creating empty list of constraints
TransModel.constraints = ConstraintList()

# adding constraint for production capacity
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for j in col_indicies) <= warehouse_cap[i]) for i in row_indicies]
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for i in row_indicies) == market_demand[j]) for j in col_indicies]

# printing out pyomo model
TransModel.pprint()

4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
       

In [41]:
# installing cbc solver in collab session
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')
result = optCBC.solve(TransModel)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [42]:
import pandas as pd
Results = pd.DataFrame({'M1':[TransModel.x[i,0].value for i in row_indicies],
                        'M2':[TransModel.x[i,1].value for i in row_indicies],
                        'M3':[TransModel.x[i,2].value for i in row_indicies],
                        'M4':[TransModel.x[i,3].value for i in row_indicies],
                        'M5':[TransModel.x[i,4].value for i in row_indicies],
                        'M6':[TransModel.x[i,5].value for i in row_indicies],
                        'M7':[TransModel.x[i,6].value for i in row_indicies],
                        'M8':[TransModel.x[i,7].value for i in row_indicies]})

In [43]:
Results


,M1,M2,M3,M4,M5,M6,M7,M8
0,0.0,185.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,350.0,0.0,0.0,0.0
2,0.0,0.0,375.0,0.0,30.0,0.0,0.0,0.0
3,0.0,200.0,0.0,0.0,0.0,80.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,20.0,0.0,85.0
5,0.0,0.0,0.0,70.0,0.0,0.0,5.0,0.0
6,175.0,15.0,175.0,0.0,0.0,0.0,60.0,0.0
7,0.0,0.0,0.0,0.0,270.0,0.0,0.0,0.0


In [45]:
TransModel.Objective()

1166465.0

In [46]:
#3.3
# number of variable representing warehouse (M=8) and markets (N=8)
M, N = 8,8

# since variable are two dimensional ie., x_{ij}, therefore both row and column indicies are defined below
row_indicies = np.arange(M)
col_indicies = np.arange(N)

# matrix of cost cofficient
cost = np.array([[627, 617, 1270, 2072, 1415,2943, 1064, 4265], [100000, 1432, 351, 3372, 237, 3373, 1456, 5059], [609, 1943, 77, 3365, 298, 3021, 1484, 4803], [2143, 671, 3876, 537, 1902, 1941, 1682, 3348],
                 [2742, 2089, 1931, 2199, 1226, 739, 691, 100000],[2053, 860, 2995, 100000, 2069, 2065, 993, 3342],[656, 817, 1245, 2141, 1534, 3351, 1137, 3887],[195, 1862, 1331, 2921, 402, 2825, 563, 4441]])

warehouse_cap = np.array([185,350,405,280,105,75,425,270])
market_demand = np.array([175, 400, 550, 70, 650, 100, 65, 85])
# defining an empty model using pyomo
TransModel = ConcreteModel()

# defining varibles used in model x_{ij} (in total 8*8 = 64 variables)
TransModel.x = Var(row_indicies, col_indicies, domain= NonNegativeReals)

# setting up the objective function
TransModel.Objective = Objective(expr = sum(cost[i][j]*TransModel.x[i,j] for i in row_indicies for j in col_indicies), sense = minimize)

In [47]:
# below we define constraints based on available demand at hub and capacity at plant
# creating empty list of constraints
TransModel.constraints = ConstraintList()

# adding constraint for production capacity
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for j in col_indicies) <= warehouse_cap[i]) for i in row_indicies]
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for i in row_indicies) == market_demand[j]) for j in col_indicies]

# printing out pyomo model
TransModel.pprint()


4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
       

In [48]:
8# installing cbc solver in collab session
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')
result = optCBC.solve(TransModel)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [49]:
import pandas as pd
Results = pd.DataFrame({'M1':[TransModel.x[i,0].value for i in row_indicies],
                        'M2':[TransModel.x[i,1].value for i in row_indicies],
                        'M3':[TransModel.x[i,2].value for i in row_indicies],
                        'M4':[TransModel.x[i,3].value for i in row_indicies],
                        'M5':[TransModel.x[i,4].value for i in row_indicies],
                        'M6':[TransModel.x[i,5].value for i in row_indicies],
                        'M7':[TransModel.x[i,6].value for i in row_indicies],
                        'M8':[TransModel.x[i,7].value for i in row_indicies]})

In [50]:
Results

,M1,M2,M3,M4,M5,M6,M7,M8
0,0.0,185.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,350.0,0.0,0.0,0.0
2,0.0,0.0,375.0,0.0,30.0,0.0,0.0,0.0
3,0.0,200.0,0.0,70.0,0.0,0.0,0.0,10.0
4,0.0,0.0,0.0,0.0,0.0,100.0,5.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0
6,175.0,15.0,175.0,0.0,0.0,0.0,60.0,0.0
7,0.0,0.0,0.0,0.0,270.0,0.0,0.0,0.0


In [51]:
TransModel.Objective()

1289875.0